# Does language distinguish groups of businesses/people? What in the language? Why? 

In [ ]:
# Does review comments help predict 
# Classify where the customers are from /business is located
# Mutual information, discrete correlation between variables, correlation is linear,
# mutual will pick up non-linear

# correlation (linear, one causes the other) vs relationship (mutual, knowing one can allow for us to know the other)

# didn't vs did not
# names of companies ? What nouns to get rid of ?

In [ ]:
import pandas as pd
import nltk
import math
import numpy as np
from nltk.corpus import stopwords 
from tqdm import tqdm
from collections import *
import operator
import itertools
import dill
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import math
pd.set_option('display.max_columns', 500)

# all words
# words that occur more than once (get rid of ones that may be typos)
# get rid of stop words
# get rid of symbols ? 
# generator, defaultdict, get

In [ ]:
filename = 'saveFile.pkl'
#dill.dump_session(filename)
#dill.load_session(filename)

# Read files

In [ ]:
#data = pd.read_csv('business_text_stripped.csv', delimiter = ',')
businesses = pd.read_csv("Businesses (1).csv")
bf = pd.read_csv('business_text_stripped.csv')
toronto_map = gpd.read_file('Neighbourhoods.geojson')
stop_words =  set(stopwords.words('english')) 

In [ ]:
stop_words

# Functions

In [ ]:
# create a dictionary of all words as long as they have more than 1 letter
def everyWord(file, stop_words):
    everyWord_counter = Counter()
    everyWord_set = set()
    for comment in tqdm(file['1'].values):
        comment = [t.lower() for t in comment.split(" ") if len(t) > 1]
        for c in comment:
            if c not in stop_words:
                everyWord_set.add(c)
        everyWord_counter.update(comment)
    return everyWord_counter

In [ ]:
# filter out words that are digits, or appear less than 10 times
def topktermFreq(everyWord, stop_words, k):
    AllWords_stopDigits = Counter(everyWord)
       
    # create an ordered dictionary by decreasing term frequency
    sorted_dict = OrderedDict(sorted(AllWords_stopDigits.items(), key=operator.itemgetter(1), reverse=True))
    top10000termFreq = dict(itertools.islice(sorted_dict.items(), k))
    return top10000termFreq

In [ ]:
def top10kdocFreq(file, topkTF):
    # iterate through the comments
    # make a set of words in the comments (so it only appears once)
    top10000docFreq = defaultdict(int)
    for comment in tqdm(file['1'].values):
        comment = set(t.lower() for t in comment.split(" ") if len(t) > 1)
        for word in comment:
            if word in top10kTF.keys():
                top10000docFreq[word] += 1
            else:
                continue
    return top10000docFreq

In [ ]:
def TFIDF(file, top10kTF, top10kDF, stop_words):
    TFIDF = np.zeros((32022,10000))
    # for each of the comments (rows)
    row_index = 0
    # loop through each of the comments in the 
    for comment in tqdm(file['1'].values):
        comment = [t.lower() for t in file.iloc[row_index][2].split(" ") if len(t) > 1 and if t not in stop_words]
        comment_counter = Counter(comment)
        col_index = 0
        # loop through the top 10k words
        for word in top10kTF.keys():
            if word in comment_counter.keys():
                # equation
                tf = comment_counter[word]/len(comment)
                idf = math.log(32022/top10kDF[word])
                TFIDF[row_index][col_index] = tf*idf
                col_index += 1
            else:
                col_index += 1
        row_index += 1
    return TFIDF

# --- Classifier ---

# cos_sim: 32,022 x 32,022 matrix of dot prods

In [ ]:
# This was tested against Alex's code and was proven to work, therefore it is ok to test without testing this function
def existing_neighbourhood_dictionary(businesses_file, neighbourhoods_file):
    id=[]
    latitude=[]
    longitude=[]

    for x in range(businesses_file.shape[0]):
        id.append(businesses_file.iloc[x,13])
        longitude.append(businesses_file.iloc[x,30])
        latitude.append(businesses_file.iloc[x,31])
    
    df = pd.DataFrame(
        {'ID': id,
         'Latitude': latitude,
         'Longitude': longitude})

    gdf = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Longitude, df.Latitude))
    
    business_neighbourhood = {}
    b_names = businesses_file.iloc[:,13]
    for key in b_names:
        business_neighbourhood[key] = float('nan')
        
    for i in range(gdf.shape[0]): #for businesses
        for j in range(neighbourhoods_file.shape[0]): # for neighbourhood
            if (neighbourhoods_file.loc[j, 'geometry']).contains(gdf.iloc[i,3]) == True:
                business_neighbourhood[businesses_file.iloc[i,13]] = neighbourhoods_file.iloc[j,6]
    return business_neighbourhood

In [ ]:
existing_neighbourhood_dictionary(businesses, toronto_map)

In [ ]:
def cos_sim(tfidf):
    similarities = np.ones((len(tfidf),len(tfidf)))
    for i in range(len(tfidf)):
        a = tfidf[i]
        for j in range(len(tfidf)):
            if i == j:
                continue
            else:
                b = tfidf[j]
                similarities[i][j] = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return similarities

In [ ]:
def kNearestNeighbours(comments_file, businesses_file, existing_neighbourhoods_dict, cos_sim, k): #want to specify the size
    #return the kNearestNeighbours dictionary
    kNearestNeighbours = {}
    
    # Have a dictionary stating the business and the neighbourhood, for ex. {2: 'A', 3: 'C', 6: 'C'}
    kNeighbours = existing_neighbourhood_dictionary(businesses_file, neighbourhoods_file)
    
    for row in tqdm(file['0'].values):
        unordered = {}
        #first column is the angle between (cos_sim), second column is the label
        for col in range(len(cos_sim)):
            if cos_sim[row][col] != 1:
                unordered[businesses_file.iloc[col,13]] = cos_sim[row][col]
        # sort the dictionary by decreasing values (it does not include the value of 1)
        ordered = OrderedDict(sorted(unordered.items(), key = operator.itemgetter(1), reverse = True))
        # take the top k number in the dictionary
        kNearest = dict(itertools.islice(ordered.items(), k))
               
        # Start adding up the results
        tally = dict.fromkeys(set(kNeighbours.values()), 0)
        for business in kNearest.keys():
            tally[kNeighbours[business]] += 1/k
        
        tallyOrdered = dict(itertools.islice(OrderedDict(sorted(tally.items(), key = operator.itemgetter(1), reverse = True)).items(), 1))
          
        kNearestNeighbours[business] = list(tallyOrdered.keys())[0]
        
    return kNearestNeighbours
        
        
# loop through rows of cos_sim (represent each business)
# create a k-nearest neighbour dictionary, with zero keys and none for values
# loop through the columns of the cos_sim
# if the value at the [row][column] index is 1, then continue, it was the dot product with itself
# else, remove the min key, add the current one, and update the value

# once we have the k-nearest neighbour dictionary, we can start adding up the results
# create a set of the results
# make a tally dictionary with 0 values and keys of sets
# loop through the labels (values from the previous k-nearest dictionary)
# += 1/k for frequency
# sorted dict to make the dictionary ordered by the highest to lowest freq

# add this sorted dict to the overall dictionary 